In [10]:
import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM

import mlm
from mlm.scorers import MLMScorerPT 
from mlm.models import get_pretrained

import mxnet as mx
ctxs = [mx.cpu()]

import pathlib
import os

from dataset_orm import *
from wordbank_tasks import *

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

## NOTES

* **TODO**: verify the surgery I did to the mlm.scorers codebase to accept RoBERTa is legit
* If we want other models, we'll have to add them there, too, perhaps with a bit more work if their output format is very different
* The function below implements the very basic test. Next steps I can see us wanting to do:
    * Combine it with sentences from the real data
    * Check at least two alternative word-replacement strategies (within category, between categories)
    * Write more of a pipeline that samples words, sentences, replacement words for each sentence, and spits out scorers
* Open questions:
    * How do we measure how well the model did? Rank of the correct sentence? NLL difference from correct and other best-performing sentence? Both? 

In [7]:
def scorer_from_transformers_checkpoint(checkpotint_name, contexts):
    tokenizer = AutoTokenizer.from_pretrained(checkpotint_name)
    model = AutoModelForMaskedLM.from_pretrained(checkpotint_name)
    return MLMScorerPT(model, None, tokenizer, contexts)

roberta_scorer = scorer_from_transformers_checkpoint('nyu-mll/roberta-base-100M-1', ctxs)
bert_scorer = scorer_from_transformers_checkpoint('bert-base-uncased', ctxs)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
DB_FILE = 'wordbank.db'
DB_PATH = pathlib.Path(os.getcwd()).parent.absolute() / 'data' / DB_FILE
engine = create_engine(f'sqlite:///{DB_PATH}')
Session = sessionmaker(bind=engine)

In [19]:
session = Session()
apple_word = session.query(WordbankWord).filter(WordbankWord.word == 'apple').one()
discriminative_task_single_word(
    Session, apple_word, 5, 5,
    [bert_scorer, roberta_scorer],
    smallest_nll_criterion)

/Users/guydavidson/opt/anaconda3/envs/torch/lib/python3.8/site-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[23, 26, 29, 32, 35, 38, 41, 44]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
/Users/guydavidson/opt/anaconda3/envs/torch/lib/python3.8/site-packages/gluonnlp/data/sampler.py:353: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[25, 29, 33, 37, 41, 45]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %


[[True, False, False, False, False], [True, False, False, True, False]]